1. Import Library & Konfigurasi
2. Load & Split Dataset
3. Data Loader & Preprocessing
4. Model 1: CNN Base (Non-Pretrained)
5. Evaluasi CNN Base
6. Model 2: Transfer Learning (ResNet50)
7. Evaluasi ResNet50
8. Model 3: Transfer Learning (MobileNetV2)
9. Evaluasi MobileNetV2
10. Perbandingan Hasil Model

In [4]:
import os, shutil, random
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import classification_report, confusion_matrix

e:\ModulS7\UAP\.venv\Lib\site-packages\keras\src\export\tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


SPLIT DATASET

In [3]:
BASE_DIR = r"E:\ModulS7\UAP"
SOURCE = os.path.join(BASE_DIR, "DatasetUAP")
TARGET = os.path.join(BASE_DIR, "data")

random.seed(42)

for cls in os.listdir(SOURCE):
    imgs = os.listdir(os.path.join(SOURCE, cls))
    random.shuffle(imgs)

    n = len(imgs)
    train = imgs[:int(0.7*n)]
    val = imgs[int(0.7*n):int(0.85*n)]
    test = imgs[int(0.85*n):]

    for split, files in zip(["train","val","test"], [train,val,test]):
        dst = os.path.join(TARGET, split, cls)
        os.makedirs(dst, exist_ok=True)
        for f in files:
            shutil.copy(
                os.path.join(SOURCE, cls, f),
                os.path.join(dst, f)
            )

DATA LOADER

In [7]:
BASE_DIR = r"E:\ModulS7\UAP"
TARGET = os.path.join(BASE_DIR, "data")

def load_data(img_size=(224,224), batch_size=32):
    train = tf.keras.preprocessing.image_dataset_from_directory(
        os.path.join(TARGET,"train"),
        image_size=img_size,
        batch_size=batch_size
    )
    val = tf.keras.preprocessing.image_dataset_from_directory(
        os.path.join(TARGET,"val"),
        image_size=img_size,
        batch_size=batch_size
    )
    test = tf.keras.preprocessing.image_dataset_from_directory(
        os.path.join(TARGET,"test"),
        image_size=img_size,
        batch_size=batch_size,
        shuffle=False
    )
    return train, val, test, train.class_names

MODEL 1 — CNN BASE (NON-PRETRAINED)

In [ ]:
train_ds, val_ds, test_ds, class_names = load_data()

model_cnn = tf.keras.Sequential([
    tf.keras.layers.Rescaling(1./255, input_shape=(224,224,3)),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(64, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(len(class_names), activation='softmax')
])

model_cnn.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

history_cnn = model_cnn.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10
)

os.makedirs("models", exist_ok=True)
model_cnn.save("models/cnn_base.keras")

Found 34844 files belonging to 7 classes.
Found 7467 files belonging to 7 classes.
Found 7468 files belonging to 7 classes.
Epoch 1/10


e:\ModulS7\UAP\.venv\Lib\site-packages\keras\src\layers\preprocessing\data_layer.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


 194/1089 ━━━━━━━━━━━━━━━━━━━━ 3:59 268ms/step - accuracy: 0.2741 - loss: 2.7179

EVALUASI

In [ ]:
y_true = np.concatenate([y for _, y in test_ds])
y_pred = np.argmax(model_cnn.predict(test_ds), axis=1)

print(classification_report(y_true, y_pred, target_names=class_names))

In [ ]:
cm = confusion_matrix(y_true, y_pred)

plt.figure(figsize=(6,5))
sns.heatmap(cm, annot=True, fmt="d",
            xticklabels=class_names,
            yticklabels=class_names)
plt.title("CNN Base - Confusion Matrix")
plt.show()

MODEL 2 — RESNET50

In [ ]:
base_model = tf.keras.applications.ResNet50(
    weights="imagenet",
    include_top=False,
    input_shape=(224,224,3)
)

base_model.trainable = False  # WAJIB

inputs = tf.keras.Input(shape=(224,224,3))
x = tf.keras.applications.resnet.preprocess_input(inputs)
x = base_model(x, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(128, activation='relu')(x)
outputs = tf.keras.layers.Dense(len(class_names), activation='softmax')(x)

model_resnet = tf.keras.Model(inputs, outputs)

Compile & Train

In [ ]:
model_resnet.compile(
    optimizer=tf.keras.optimizers.Adam(1e-4),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

history_resnet = model_resnet.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10
)

os.makedirs("models", exist_ok=True)
model_resnet.save("models/resnet50.keras")

Evaluasi ResNet50

In [ ]:
y_true = np.concatenate([y for _, y in test_ds])
y_pred = np.argmax(model_resnet.predict(test_ds), axis=1)

print("=== RESNET50 ===")
print(classification_report(y_true, y_pred, target_names=class_names))

In [ ]:
cm = confusion_matrix(y_true, y_pred)

plt.figure(figsize=(6,5))
sns.heatmap(cm, annot=True, fmt="d",
            xticklabels=class_names,
            yticklabels=class_names)
plt.title("ResNet50 - Confusion Matrix")
plt.show()

MODEL 3 — MOBILENETV2

In [ ]:
base_model = tf.keras.applications.MobileNetV2(
    weights="imagenet",
    include_top=False,
    input_shape=(224,224,3)
)

base_model.trainable = False

inputs = tf.keras.Input(shape=(224,224,3))
x = tf.keras.applications.mobilenet_v2.preprocess_input(inputs)
x = base_model(x, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(128, activation='relu')(x)
outputs = tf.keras.layers.Dense(len(class_names), activation='softmax')(x)

model_mobilenet = tf.keras.Model(inputs, outputs)

Compile & Train

In [ ]:
model_mobilenet.compile(
    optimizer=tf.keras.optimizers.Adam(1e-4),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

history_mobilenet = model_mobilenet.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10
)

model_mobilenet.save("models/mobilenetv2.keras")

Evaluasi MobileNetV2

In [ ]:
y_pred = np.argmax(model_mobilenet.predict(test_ds), axis=1)

print("=== MOBILENETV2 ===")
print(classification_report(y_true, y_pred, target_names=class_names))

GRAFIK LOSS & ACCURACY

In [ ]:
def plot_history(history, title):
    plt.figure(figsize=(10,4))

    plt.subplot(1,2,1)
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title(f'{title} Accuracy')
    plt.legend(['Train','Val'])

    plt.subplot(1,2,2)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title(f'{title} Loss')
    plt.legend(['Train','Val'])

    plt.show()

In [ ]:
plot_history(history_resnet, "ResNet50")
plot_history(history_mobilenet, "MobileNetV2")